In [1]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

def ollama_chat_client():
    return OpenAIChatClient(
        api_key="ollama",
        model_id="gemma3:1b",
        base_url="http://localhost:11434/v1"  # la URL de tu servidor Ollama
    )



### Agents

In [2]:
agent = ChatAgent(
    chat_client=ollama_chat_client(),
    instructions="You are a helpful assistant.",
    name="LocalOllamaAgent"
)

agent_response = await agent.run("Tell me a jode about PwC")
agent_response.text


"Okay, let's dive into PwC – PwC: A Global Consulting Firm. Here's a breakdown of what they are, what they do, and why they're a significant player in the world:\n\n**What is PwC?**\n\nPwC is a multinational professional services network.  Simply put, they’re a company that offers a huge range of services to businesses, governments, and nonprofits. They aren’t just one firm; they are a network of independent firms, each specializing in a particular industry or area of expertise.  Think of it like a collection of talented consultants and specialists.\n\n**Key Areas of Work:**\n\nPwC's work is incredibly diverse and spans a massive range of industries. Here’s a look at some of their key areas:\n\n* **Audit & Assurance:** They provide assurance services to companies, ensuring they comply with regulations and deliver reliable financial information.\n* **Consulting:** This is a HUGE part of what they do. They help businesses:\n    * **Strategy:** Develop and execute long-term plans, identif

In [3]:
from datetime import datetime
from agent_framework import ai_function

@ai_function(name="time_tool", description="Retrieves the actual time")
def get_time() -> None:
    time = datetime.now()
    return f"The hour is {time.hour}:{time.minute}"

In [6]:
time_agent = ChatAgent(
    chat_client=ollama_chat_client(),
    instructions="You are a helpful assistant.",
    name="LocalOllamaAgent"
)

In [7]:
time_agent_response = await time_agent.run("What hour it is?")
time_agent_response.text

'It’s 2:44 PM PDT.'

### Workflows

In [8]:
import asyncio
from typing_extensions import Never
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor, Executor, handler

In [9]:
class UpperCase(Executor):
    def __init__(self, id: str):
        super().__init__(id=id)

    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        result = text.upper()
        await ctx.send_message(result)

In [10]:
@executor(id="reverse_text_executor")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    result = text[::-1]
    await ctx.yield_output(result)

In [11]:
upper_case = UpperCase(id="upper_case_executor")

workflow = (
    WorkflowBuilder()
    .add_edge(upper_case, reverse_text)
    .set_start_executor(upper_case)
    .build()
)

In [12]:
async def main():
    async for event in workflow.run_stream("hello world"):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")


In [13]:
await main()

Event: WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
Event: ExecutorInvokedEvent(executor_id=upper_case_executor, data=None)
Event: ExecutorCompletedEvent(executor_id=upper_case_executor, data=None)
Event: ExecutorInvokedEvent(executor_id=reverse_text_executor, data=None)
Event: WorkflowOutputEvent(data=DLROW OLLEH, source_executor_id=reverse_text_executor)
Workflow completed with result: DLROW OLLEH
Event: ExecutorCompletedEvent(executor_id=reverse_text_executor, data=None)
Event: WorkflowStatusEvent(state=WorkflowRunState.IDLE, data=None, origin=WorkflowEventSource.FRAMEWORK)


#### Workflow with agents

In [14]:
writer = ChatAgent(
    instructions=(
        "You are a concise copywriter. Provide a single, punchy marketing sentence based on the prompt."
    ),
    name="writer",
    chat_client=ollama_chat_client()
)

reviewer = ChatAgent(
    instructions=(
        "You are a thoughtful reviewer. Give brief feedback on the previous assistant message."
    ),
    name="reviewer",
    chat_client=ollama_chat_client()
)

In [15]:
from agent_framework import SequentialBuilder

# 2) Build sequential workflow: writer -> reviewer
workflow = SequentialBuilder().participants([writer, reviewer]).build()

In [19]:
from agent_framework import ChatMessage
from agent_framework import Role
from typing import Any

events = []
async for event in workflow.run_stream("Write a tagline for a budget-friendly eBike."):
    events.append(event)


print("===== Final Conversation =====")
messages: list[ChatMessage] | Any = events
for i, msg in enumerate(messages, start=1):
    print(f"{'-' * 60}\n{i:02d} {msg}")

===== Final Conversation =====
------------------------------------------------------------
01 WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
------------------------------------------------------------
02 WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
------------------------------------------------------------
03 ExecutorInvokedEvent(executor_id=input-conversation, data=None)
------------------------------------------------------------
04 ExecutorCompletedEvent(executor_id=input-conversation, data=None)
------------------------------------------------------------
05 ExecutorInvokedEvent(executor_id=writer, data=None)
------------------------------------------------------------
06 AgentRunUpdateEvent(executor_id=writer, messages=Okay)
------------------------------------------------------------
07 AgentRunUpdateEvent(executor_id=writer, messages=,)
---------------------------------------------------------